In [1]:
import pandas as pd
dados = pd.read_csv('../marketing_investimento.csv')
dados

,idade,estado_civil,escolaridade,inadimplencia,saldo,fez_emprestimo,tempo_ult_contato,numero_contatos,aderencia_investimento
0,45,casado (a),superior,nao,242,nao,587,1,sim
1,42,casado (a),medio,nao,1289,nao,250,4,sim
2,23,solteiro (a),superior,nao,363,nao,16,18,nao
3,58,divorciado (a),superior,nao,1382,nao,700,1,sim
4,50,casado (a),medio,nao,3357,nao,239,4,sim
...,...,...,...,...,...,...,...,...,...
1263,52,solteiro (a),superior,nao,83,nao,1223,6,sim
1264,35,solteiro (a),superior,nao,5958,nao,215,1,sim
1265,30,solteiro (a),superior,nao,-477,sim,1532,2,sim
1266,42,casado (a),superior,nao,2187,nao,525,3,sim


In [2]:
x = dados.drop('aderencia_investimento', axis=1)
y = dados['aderencia_investimento']

Os algoritmos de machine learning não compreendem dados no formato de texto, portanto devemos fazer transformações nos dados para o formato numérico para que o algoritmo consiga compreender as informações. Essa transformação precisa ser feita de forma que não altere a informação original do conjunto de dados, portanto não basta simplesmente alterar os valores para valores numéricos aleatórios.

Usaremos o **OneHotEncoder** para transformar valores textuais em zeros e uns, usaremos também o **make_column_transformer** para fazer em mais de uma coluna, exemplo:

**id** | **estado civil**
:--: | :-----------:
0 | casado
1 | solteiro
2 | casado
3 | divorciado

==>

id | casado | solteiro | divorciado
:--: | :------: | :------: | :---------:
0 | 1 | 0 | 0
1 | 0 | 1 | 0
2 | 1 | 0 | 0
3 | 0 | 0 | 1

In [3]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [4]:
colunas = x.columns #Para ter o backup das colunas que faremos alterações

In [5]:
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #Se a coluna tiver dois valores ele não vai criar duas colunas, vai fazer aquela ser 0 ou 1
    ['estado_civil', 'escolaridade', 'inadimplencia', 'fez_emprestimo'] #Colunas que desejamos fazer as alterações
),
    remainder='passthrough', #Indica que não queremos excluir as colunas que não serão alteradas
    sparse_threshold=0) 

In [6]:
x = one_hot.fit_transform(x)

In [ ]:
one_hot.get_feature_names_out(colunas) #Para retornar qual nome novo das colunas com base nas colunas originais

array(['onehotencoder__estado_civil_casado (a)',
       'onehotencoder__estado_civil_divorciado (a)',
       'onehotencoder__estado_civil_solteiro (a)',
       'onehotencoder__escolaridade_fundamental',
       'onehotencoder__escolaridade_medio',
       'onehotencoder__escolaridade_superior',
       'onehotencoder__inadimplencia_sim',
       'onehotencoder__fez_emprestimo_sim', 'remainder__idade',
       'remainder__saldo', 'remainder__tempo_ult_contato',
       'remainder__numero_contatos'], dtype=object)

In [9]:
pd.DataFrame(x, columns =  one_hot.get_feature_names_out(colunas))

,onehotencoder__estado_civil_casado (a),onehotencoder__estado_civil_divorciado (a),onehotencoder__estado_civil_solteiro (a),onehotencoder__escolaridade_fundamental,onehotencoder__escolaridade_medio,onehotencoder__escolaridade_superior,onehotencoder__inadimplencia_sim,onehotencoder__fez_emprestimo_sim,remainder__idade,remainder__saldo,remainder__tempo_ult_contato,remainder__numero_contatos
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,45.0,242.0,587.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,42.0,1289.0,250.0,4.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,23.0,363.0,16.0,18.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,58.0,1382.0,700.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,50.0,3357.0,239.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1263,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,52.0,83.0,1223.0,6.0
1264,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,35.0,5958.0,215.0,1.0
1265,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,30.0,-477.0,1532.0,2.0
1266,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,42.0,2187.0,525.0,3.0


# Transformando a variável alvo (y)

O Scikit-Learn possui uma função chamada Label Encoder, exclusiva para fazer a transformação da variável alvo. A documentação da biblioteca Scikit-Learn diz que esse transformador deve ser usado para codificar valores alvo (ou seja, o y), e não a entrada x, que são as variáveis explicativas.

Estamos fazendo a mesma coisa que fizemos acima só que de maneira mais simplificada pois temos apenas uma coluna

In [11]:
from sklearn.preprocessing import LabelEncoder

In [13]:
label_encoder = LabelEncoder()

In [14]:
y = label_encoder.fit_transform(y)